In [1]:
"""
this jupyter is used for analyzing and drawing the topics & sentiments inferred by OneModel using 2 months vaccine-related tweets 
"""
import sys
sys.path.append("../") # go to parent dir
from codebase.topic_utilities import dtm_csv_to_pd_df, calculate_topic_ratios

import numpy as np
import pandas as pd

In [2]:
# fixed params
corpora_path = "../corpora/"
model_path = "../models/"

# params
fileTag_list = ["First-and-SecondWeek-Tweets-Rolling","Third-and-FourthWeek-Tweets-Rolling",\
    "Fifth-and-SixthWeek-Tweets-Rolling", "Seventh-and-EighthWeek-Tweets-Rolling"]

start_list = ["2020-12-16","2020-12-31", "2021-01-15", "2021-01-30"]
end_list = ["2020-12-31", "2021-01-15", "2021-01-30", "2021-02-14"]

num_topics = 50
model_suffix = "-{}topics".format(num_topics)

# Generate Dated Tweets with Max Topic

In [ ]:
def return_dated_topic_frame(dtm, metadata, start, end):
    df_max = dtm.idxmax(axis=1).to_frame().reset_index()
    df_max.columns = ["position_index", "max_topic"]

    # drop duplicated tweets 
    metadata.drop_duplicates(subset="id_str", inplace=True)
    # merge with metadata for fecthing date
    df = df_max.merge(metadata, on="position_index", how="right")
    df['preDate'] = pd.to_datetime(df['created_time'])
    # format the Date column for grouping conveniently
    df["Date"] = df["preDate"].dt.strftime("%Y-%m-%d")
    # drop position_index, created_time, preDate
    df = df.drop(['position_index', 'created_time', 'preDate'], axis=1)
    return df.reset_index()

In [ ]:
%%time
list_of_dfs = []
for fileTag, start, end in zip(fileTag_list, start_list, end_list):
    
    print("Start {} from {} to {}.".format(fileTag, start, end))

    metadata_filename = "{}{}-Meta.csv".format(corpora_path, fileTag)
    # Read back data from csv files
    metadata = pd.read_csv(metadata_filename)
    doc_topic_matrix_filename = "{}{}{}-dtm.csv".format(model_path, fileTag, model_suffix)
    dtm = dtm_csv_to_pd_df(doc_topic_matrix_filename)
    list_of_dfs.append(return_dated_topic_frame(dtm, metadata, start, end))

In [ ]:
# concat_dfs = pd.concat(list_of_dfs)
# concat_dfs.to_pickle("../corpora/distinct_tweets/dated_distinct_tweets_dtm.pickle")